# Data Pipelines
#### Silicon Slayers

Source: [Databricks](https://docs.databricks.com/en/getting-started/data-pipeline-get-started.html#:~:text=A%20data%20pipeline%20implements%20the,data%20that%20users%20can%20consume)



### What is a data pipeline? 
Data pipelines are a process for moving data from sources (like a UI/Application) to targets (like a data warehouse). 

Data is extracted from the source, "flows" through the pipeline and neccesary transformations, and is loaded into the target. 

A key feature of data pipelines is automation; data can be loaded in batches or in (near) real time. 

### What is ETL? 
A common abbreviation is ETL, which stands for Extract, Transform, Load. ELT is also becoming more popular with cloud warehouses. 

### What are common ETL Tools? 
- Apache (Airflow, Kafka, Spark)  
- dbt (Data Build Tool)  
- Informatica  

- AWS Glue  
- Google Cloud Dataflow  
- Microsoft Azure Data Factory  


%md
For our pipeline, we are going to break out the different steps into different notebooks. 

Why keep them seperate? This allows our code to be more modular and, in real organizations, for different teams to work on seperate parts of the process. 

In [ ]:
from pyspark.sql.types import DoubleType, IntegerType, StringType, StructType, StructField

# Define variables used in the code below
file_path = "/databricks-datasets/songs/data-001/"
table_name = "raw_song_data"
checkpoint_path = "/tmp/pipeline_get_started/_checkpoint/song_data"

schema = StructType(
  [
    StructField("artist_id", StringType(), True),
    StructField("artist_lat", DoubleType(), True),
    StructField("artist_long", DoubleType(), True),
    StructField("artist_location", StringType(), True),
    StructField("artist_name", StringType(), True),
    StructField("duration", DoubleType(), True),
    StructField("end_of_fade_in", DoubleType(), True),
    StructField("key", IntegerType(), True),
    StructField("key_confidence", DoubleType(), True),
    StructField("loudness", DoubleType(), True),
    StructField("release", StringType(), True),
    StructField("song_hotnes", DoubleType(), True),
    StructField("song_id", StringType(), True),
    StructField("start_of_fade_out", DoubleType(), True),
    StructField("tempo", DoubleType(), True),
    StructField("time_signature", DoubleType(), True),
    StructField("time_signature_confidence", DoubleType(), True),
    StructField("title", StringType(), True),
    StructField("year", IntegerType(), True),
    StructField("partial_sequence", IntegerType(), True)
  ]
)

(spark.readStream
  .format("cloudFiles")
  .schema(schema)
  .option("cloudFiles.format", "csv")
  .option("sep","\t")
  .load(file_path)
  .writeStream
  .option("checkpointLocation", checkpoint_path)
  .trigger(availableNow=True)
  .toTable(table_name)
)

Out[3]: <pyspark.sql.streaming.query.StreamingQuery at 0x7fc515071ca0>

In [ ]:
%sql
CREATE OR REPLACE TABLE
    artists (
        artist_id STRING,
        artist_lat DOUBLE,
        artist_long DOUBLE,
        artist_location STRING,
        artist_name STRING,
        processed_time TIMESTAMP
        );

INSERT INTO artists
    SELECT DISTINCT
        artist_id,
        artist_latitude,
        artist_longitude,
        artist_location,
        artist_name,
        current_timestamp()
    FROM raw_song_data
    WHERE artist_id IS NOT NULL;

In [ ]:
%sql 
CREATE OR REPLACE TABLE
    songs (
        song_id STRING,
        title STRING,
        artist_id STRING,
        year INT,
        duration DOUBLE,
        song_popularity DOUBLE,
        loudness DOUBLE,
        processed_time TIMESTAMP
        );

INSERT INTO songs
    SELECT DISTINCT
        song_id,
        title,
        artist_id,
        year,
        duration,
        song_hotnes,
        loudness,
        current_timestamp()
    FROM raw_song_data
    WHERE song_id IS NOT NULL;